# Imoprt CVS Information Page

Purpose of this Project: Extract Vaccine appointment info from the CVS. CVS data team has provide the links of appointment, slot and schedule. For more information please refer to https://developer.cvshealth.com/covid19/cdc/bulk-publish

Key Point:
Json File
Ndjson File


In [2]:
import requests
import json
import pandas as pd
import ndjson  ## Location Schedule and Slot are the Ndjson format
from datetime import datetime
from datetime import date

In [3]:
re=requests.get("https://www.cvs.com/immunizations/inventory/data/$bulk-publish")

In [4]:
# Check the status
re.status_code

200

In [10]:
re.json()

{'transactionTime': '2021-05-21T10:26:09.534Z',
 'request': 'https://www.cvs.com/immunizations/inventory/data/$bulk-publish',
 'output': [{'type': 'Location',
   'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_05212021032609.ndjson'},
  {'type': 'Schedule',
   'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05212021032609.ndjson'},
  {'type': 'Slot',
   'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_05212021032609.ndjson'}],
 'error': []}

In [11]:
re.json()['output']

[{'type': 'Location',
  'url': 'https://www.cvs.com/immunizations/inventory/data/location_file_05212021032609.ndjson'},
 {'type': 'Schedule',
  'url': 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05212021032609.ndjson'},
 {'type': 'Slot',
  'url': 'https://www.cvs.com/immunizations/inventory/data/slot_file_05212021032609.ndjson'}]

In [12]:
Type=[]
Url=[]
for i in re.json()['output']:
    Type.append(i['type'])
    Url.append(i['url'])

In [13]:
Type

['Location', 'Schedule', 'Slot']

In [14]:
Url

['https://www.cvs.com/immunizations/inventory/data/location_file_05212021032609.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/schedule_file_05212021032609.ndjson',
 'https://www.cvs.com/immunizations/inventory/data/slot_file_05212021032609.ndjson']

In [15]:
df=pd.DataFrame(Url,columns=['Url'],index=Type)
#set Type as index

In [16]:
df.loc['Location']["Url"]  # becasue the type is Index

'https://www.cvs.com/immunizations/inventory/data/location_file_05212021032609.ndjson'

In [17]:
df.loc['Schedule']["Url"]

'https://www.cvs.com/immunizations/inventory/data/schedule_file_05212021032609.ndjson'

In [18]:
df.loc['Slot']['Url']

'https://www.cvs.com/immunizations/inventory/data/slot_file_05212021032609.ndjson'

In [19]:
requests.get(df.loc['Location']["Url"])

<Response [200]>

# Import

In [20]:
Location_re=requests.get(df.loc['Location']["Url"])
# there are ndjson file
#Location_re.json()     will return no value, so I used .text without .json()
Schedule_re=requests.get(df.loc['Schedule']["Url"])
Slot_re=requests.get(df.loc['Slot']["Url"])

In [ ]:
#Location_re.text

## Convert String to Ndjson Format

### Location

In [ ]:
ndjson.loads(Location_re.text)[0]
#The ndjson file are different from json file, the boundary is ",", but also be careful of the ","

In [207]:
ndjson.loads(Location_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'identifier', 'name', 'telecom', 'address'])

In [15]:
ndjson.loads(Location_re.text)[0]['telecom']
#type(ndjson.loads(Location_re.text)[0]['telecom'])  ## It is the list

[{'system': 'phone', 'value': '888-607-4287'},
 {'system': 'url',
  'value': 'https://www.cvs.com/immunizations/covid-19-vaccine?cid=oc_vacfnd_cvd'}]

In [16]:
ndjson.loads(Location_re.text)[0]['telecom'][0]
#type(ndjson.loads(Location_re.text)[0]['telecom'][0]) ## It is the dict

{'system': 'phone', 'value': '888-607-4287'}

In [17]:
ndjson.loads(Location_re.text)[0]['telecom'][0]['value']

'888-607-4287'

In [65]:
ndjson.loads(Location_re.text)[0]['address']['line'][0]

'581 HIGHLAND AVE.'

### Schedule

In [211]:
ndjson.loads(Schedule_re.text)[0]

{'resourceType': 'Schedule',
 'id': 'C250B470EE32ED97E0536CF0D90A07A9',
 'serviceType': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/service-type',
     'code': '57',
     'display': 'Immunization'},
    {'system': 'http://fhir-registry.smarthealthit.org/StructureDefinition/vaccine-dose',
     'code': 'covid19-immunization',
     'display': 'COVID-19 Immunization Appointment'}]}],
 'actor': [{'reference': 'Location/1482'}]}

In [212]:
ndjson.loads(Schedule_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'serviceType', 'actor'])

In [213]:
ndjson.loads(Schedule_re.text)[0]['actor'][0]['reference'].split('/')[1]
# In the reference: the location/9714, we only need the number after /

'1482'

In [214]:
ndjson.loads(Schedule_re.text)[0]['id']

'C250B470EE32ED97E0536CF0D90A07A9'

### Slot

In [215]:
ndjson.loads(Slot_re.text)[0]

{'resourceType': 'Slot',
 'id': 'C250B471143DED97E0536CF0D90A07A9',
 'schedule': {'reference': 'Schedule/C250B470F1F3ED97E0536CF0D90A07A9'},
 'status': 'busy',
 'start': '2021-05-17T09:00:00.000Z',
 'end': '2021-05-17T19:00:00.000Z',
 'extension': [{'url': 'http://fhir-registry.smarthealthit.org/StructureDefinition/booking-deep-link',
   'valueUrl': 'https://www.cvs.com/immunizations/covid-19-vaccine?cid=oc_vacfnd_cvd'},
  {'url': 'http://fhir-registry.smarthealthit.org/StructureDefinition/slot-capacity',
   'valueInteger': '0'}]}

In [216]:
ndjson.loads(Slot_re.text)[0].keys()

dict_keys(['resourceType', 'id', 'schedule', 'status', 'start', 'end', 'extension'])

In [217]:
ndjson.loads(Slot_re.text)[0]['schedule']['reference'].split('/')[1]

'C250B470F1F3ED97E0536CF0D90A07A9'

In [218]:
ndjson.loads(Slot_re.text)[0]['status']

'busy'

In [219]:
ndjson.loads(Slot_re.text)[0]['start']

'2021-05-17T09:00:00.000Z'

In [220]:
ndjson.loads(Slot_re.text)[0]['end']

'2021-05-17T19:00:00.000Z'

In [221]:
ndjson.loads(Slot_re.text)[0]['extension'][1]['valueInteger']

'0'

## Create For Loop to organize information of Locations

In [66]:
Locations_Info=[]
for Locations in ndjson.loads(Location_re.text):
    Detail_Info={'id':Locations['id'],
     'Line':Locations['address']['line'][0],
     'City':Locations['address']['city'],
     'State':Locations['address']['state'],
     'PostalCode':Locations['address']['postalCode'],
         }
    Locations_Info.append(Detail_Info)

In [67]:
#Locations_Info

In [68]:
Location_DF=pd.DataFrame(Locations_Info)  ## Becasue Locations_Info is the List and dic is inside of this List

In [69]:
Location_DF

,id,Line,City,State,PostalCode
0,2502,581 HIGHLAND AVE.,CHESHIRE,CT,06410
1,1521,9140 ROTHBURY DRIVE,GAITHERSBURG,MD,20886
2,4365,709 W. COLLEGE ST.,PULASKI,TN,38478
3,1087,20 FILORS LN.,STONY POINT,NY,10980
4,4984,863 BALTIMORE PIKE,GLEN MILLS,PA,19342
...,...,...,...,...,...
9732,3634,2160 LEE RD,CLEVELAND HEIGHTS,OH,44118
9733,2535,49 CLAREMONT AVE.,MONTCLAIR,NJ,07042
9734,8875,3644 EAST OLYMPIC BOULEVARD,LOS ANGELES,CA,90023
9735,675,159 N. MACDADE BLVD.,GLENOLDEN,PA,19036


In [70]:
Location_DF['State'].value_counts()
#Location_DF['State'].value_counts().sort_values()

CA    1155
TX     842
FL     838
NY     558
PA     460
MA     400
OH     385
IL     375
NC     360
VA     347
NJ     344
GA     342
IN     330
MI     297
MD     225
SC     206
MO     197
AZ     195
CT     179
AL     178
TN     158
LA     133
MN     130
NV      94
KY      81
WI      80
OK      76
HI      69
RI      61
DC      59
MS      57
WV      57
KS      54
CO      51
NH      47
WA      45
IA      38
NE      30
UT      29
ME      24
NM      24
AR      23
PR      22
OR      22
DE      20
MT      14
VT       9
ND       6
SD       3
ID       3
AK       3
WY       2
Name: State, dtype: int64

In [71]:
Location_DF.groupby(["State","City"]).count().reset_index()

,State,City,id,Line,PostalCode
0,AK,ANCHORAGE,2,2,2
1,AK,WASILLA,1,1,1
2,AL,ALABASTER,2,2,2
3,AL,ALBERTVILLE,1,1,1
4,AL,ALICEVILLE,1,1,1
...,...,...,...,...,...
4341,WV,TRIADELPHIA,1,1,1
4342,WV,WESTON,1,1,1
4343,WV,WHEELING,2,2,2
4344,WY,CASPER,1,1,1


## Create For Loop to organize information of Schedules

In [72]:
Schedule_Info=[]
for Schedules in ndjson.loads(Schedule_re.text):
    Detail_Info={'id':Schedules['id'],
                 'actor':Schedules['actor'][0]['reference'].split('/')[1]
         }
    Schedule_Info.append(Detail_Info)

In [73]:
 Schedule_Info

[{'id': 'C2D33F652DCFB21BE0536CF0D90A17F0', 'actor': '7345'},
 {'id': 'C2D33F652DD0B21BE0536CF0D90A17F0', 'actor': '1482'},
 {'id': 'C2D33F652DD1B21BE0536CF0D90A17F0', 'actor': '2416'},
 {'id': 'C2D33F652DD2B21BE0536CF0D90A17F0', 'actor': '8811'},
 {'id': 'C2D33F652DD3B21BE0536CF0D90A17F0', 'actor': '464'},
 {'id': 'C2D33F652DD4B21BE0536CF0D90A17F0', 'actor': '3011'},
 {'id': 'C2D33F652DD5B21BE0536CF0D90A17F0', 'actor': '828'},
 {'id': 'C2D33F652DD6B21BE0536CF0D90A17F0', 'actor': '10977'},
 {'id': 'C2D33F652DD7B21BE0536CF0D90A17F0', 'actor': '6663'},
 {'id': 'C2D33F652DD8B21BE0536CF0D90A17F0', 'actor': '4026'},
 {'id': 'C2D33F652DD9B21BE0536CF0D90A17F0', 'actor': '5653'},
 {'id': 'C2D33F652DDAB21BE0536CF0D90A17F0', 'actor': '5359'},
 {'id': 'C2D33F652DDBB21BE0536CF0D90A17F0', 'actor': '8830'},
 {'id': 'C2D33F652DDCB21BE0536CF0D90A17F0', 'actor': '8337'},
 {'id': 'C2D33F652DDDB21BE0536CF0D90A17F0', 'actor': '1903'},
 {'id': 'C2D33F652DDEB21BE0536CF0D90A17F0', 'actor': '6340'},
 {'id': '

In [26]:
Schedule_DF=pd.DataFrame(Schedule_Info)

In [27]:
Schedule_DF['id'].value_counts()

C2D33F653C90B21BE0536CF0D90A17F0    1
C2D33F6531B7B21BE0536CF0D90A17F0    1
C2D33F653D0CB21BE0536CF0D90A17F0    1
C2D33F654797B21BE0536CF0D90A17F0    1
C2D33F652CBBB21BE0536CF0D90A17F0    1
                                   ..
C2D33F6544FDB21BE0536CF0D90A17F0    1
C2D33F6550D4B21BE0536CF0D90A17F0    1
C2D33F653DBBB21BE0536CF0D90A17F0    1
C2D33F654FD7B21BE0536CF0D90A17F0    1
C2D33F65468DB21BE0536CF0D90A17F0    1
Name: id, Length: 9737, dtype: int64

In [29]:
Schedule_DF['actor'].value_counts()
#Schedule_DF['actor'].value_counts().max()    # 1

10074    1
2194     1
7336     1
6370     1
4844     1
        ..
16685    1
17098    1
6374     1
10376    1
3551     1
Name: actor, Length: 9737, dtype: int64

## Create For Loop to organize information of Slots

In [33]:
Slot_Info=[]
for Slots in ndjson.loads(Slot_re.text):
    Detail_Info={'id':Slots['schedule']['reference'].split('/')[1],
                 'Status':Slots['status'],
                 'Start':Slots['start'],
                 'End':Slots['end'],
                 'ValueInteger':Slots['extension'][1]['valueInteger']
         }
    Slot_Info.append(Detail_Info)

In [34]:
 Slot_Info

[{'id': 'C2D33F653191B21BE0536CF0D90A17F0',
  'Status': 'busy',
  'Start': '2021-05-22T09:00:00.000Z',
  'End': '2021-05-22T19:00:00.000Z',
  'ValueInteger': '0'},
 {'id': 'C2D33F653191B21BE0536CF0D90A17F0',
  'Status': 'busy',
  'Start': '2021-05-23T09:00:00.000Z',
  'End': '2021-05-23T19:00:00.000Z',
  'ValueInteger': '0'},
 {'id': 'C2D33F653191B21BE0536CF0D90A17F0',
  'Status': 'free',
  'Start': '2021-05-24T09:00:00.000Z',
  'End': '2021-05-24T19:00:00.000Z',
  'ValueInteger': '1'},
 {'id': 'C2D33F653191B21BE0536CF0D90A17F0',
  'Status': 'free',
  'Start': '2021-05-25T09:00:00.000Z',
  'End': '2021-05-25T19:00:00.000Z',
  'ValueInteger': '1'},
 {'id': 'C2D33F653BA5B21BE0536CF0D90A17F0',
  'Status': 'busy',
  'Start': '2021-05-22T09:00:00.000Z',
  'End': '2021-05-22T19:00:00.000Z',
  'ValueInteger': '0'},
 {'id': 'C2D33F653BA5B21BE0536CF0D90A17F0',
  'Status': 'busy',
  'Start': '2021-05-23T09:00:00.000Z',
  'End': '2021-05-23T19:00:00.000Z',
  'ValueInteger': '0'},
 {'id': 'C2D33F6

In [35]:
Slot_DF=pd.DataFrame(Slot_Info)

In [36]:
Slot_DF

,id,Status,Start,End,ValueInteger
0,C2D33F653191B21BE0536CF0D90A17F0,busy,2021-05-22T09:00:00.000Z,2021-05-22T19:00:00.000Z,0
1,C2D33F653191B21BE0536CF0D90A17F0,busy,2021-05-23T09:00:00.000Z,2021-05-23T19:00:00.000Z,0
2,C2D33F653191B21BE0536CF0D90A17F0,free,2021-05-24T09:00:00.000Z,2021-05-24T19:00:00.000Z,1
3,C2D33F653191B21BE0536CF0D90A17F0,free,2021-05-25T09:00:00.000Z,2021-05-25T19:00:00.000Z,1
4,C2D33F653BA5B21BE0536CF0D90A17F0,busy,2021-05-22T09:00:00.000Z,2021-05-22T19:00:00.000Z,0
...,...,...,...,...,...
38943,C2D33F655236B21BE0536CF0D90A17F0,free,2021-05-25T09:00:00.000Z,2021-05-25T19:00:00.000Z,1
38944,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-22T09:00:00.000Z,2021-05-22T19:00:00.000Z,1
38945,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-25T09:00:00.000Z,2021-05-25T19:00:00.000Z,1
38946,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-24T09:00:00.000Z,2021-05-24T19:00:00.000Z,1


In [37]:
Slot_DF['id'].value_counts()
#Slot_DF['id'].value_counts().max()    ## 1
#The id is the unique value

C2D33F653C90B21BE0536CF0D90A17F0    4
C2D33F6531B7B21BE0536CF0D90A17F0    4
C2D33F653916B21BE0536CF0D90A17F0    4
C2D33F653D0CB21BE0536CF0D90A17F0    4
C2D33F653AD3B21BE0536CF0D90A17F0    4
                                   ..
C2D33F654D57B21BE0536CF0D90A17F0    4
C2D33F654545B21BE0536CF0D90A17F0    4
C2D33F654A90B21BE0536CF0D90A17F0    4
C2D33F65523AB21BE0536CF0D90A17F0    4
C2D33F65468DB21BE0536CF0D90A17F0    4
Name: id, Length: 9737, dtype: int64

In [38]:
Slot_DF['Start']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['Start']]

In [39]:
Slot_DF['End']=[datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ') for x in Slot_DF['End']]

In [40]:
Slot_DF[['Status','ValueInteger']].drop_duplicates()
#Slot_DF['Status'].value_counts()
#Slot_DF['ValueInteger'].value_counts()
#Based on the result, the busy means 0, free means 1

,Status,ValueInteger
0,busy,0
2,free,1


In [41]:
Slot_DF[['Start','End']].drop_duplicates()

,Start,End
0,2021-05-22 09:00:00,2021-05-22 19:00:00
1,2021-05-23 09:00:00,2021-05-23 19:00:00
2,2021-05-24 09:00:00,2021-05-24 19:00:00
3,2021-05-25 09:00:00,2021-05-25 19:00:00


# Combine 3 DataFrames Together

The location files are the all locations of CVS
The Schedule tels service type and locations(all of them the service types are same)
The slot file tells the free or busy 

In [74]:
Location_DF

,id,Line,City,State,PostalCode
0,2502,581 HIGHLAND AVE.,CHESHIRE,CT,06410
1,1521,9140 ROTHBURY DRIVE,GAITHERSBURG,MD,20886
2,4365,709 W. COLLEGE ST.,PULASKI,TN,38478
3,1087,20 FILORS LN.,STONY POINT,NY,10980
4,4984,863 BALTIMORE PIKE,GLEN MILLS,PA,19342
...,...,...,...,...,...
9732,3634,2160 LEE RD,CLEVELAND HEIGHTS,OH,44118
9733,2535,49 CLAREMONT AVE.,MONTCLAIR,NJ,07042
9734,8875,3644 EAST OLYMPIC BOULEVARD,LOS ANGELES,CA,90023
9735,675,159 N. MACDADE BLVD.,GLENOLDEN,PA,19036


In [75]:
Schedule_DF

,id,actor
0,C2D33F652DCFB21BE0536CF0D90A17F0,7345
1,C2D33F652DD0B21BE0536CF0D90A17F0,1482
2,C2D33F652DD1B21BE0536CF0D90A17F0,2416
3,C2D33F652DD2B21BE0536CF0D90A17F0,8811
4,C2D33F652DD3B21BE0536CF0D90A17F0,464
...,...,...
9732,C2D33F655336B21BE0536CF0D90A17F0,3634
9733,C2D33F655337B21BE0536CF0D90A17F0,2535
9734,C2D33F655338B21BE0536CF0D90A17F0,8875
9735,C2D33F655339B21BE0536CF0D90A17F0,675


In [76]:
Slot_DF

,id,Status,Start,End,ValueInteger
0,C2D33F653191B21BE0536CF0D90A17F0,busy,2021-05-22 09:00:00,2021-05-22 19:00:00,0
1,C2D33F653191B21BE0536CF0D90A17F0,busy,2021-05-23 09:00:00,2021-05-23 19:00:00,0
2,C2D33F653191B21BE0536CF0D90A17F0,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1
3,C2D33F653191B21BE0536CF0D90A17F0,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
4,C2D33F653BA5B21BE0536CF0D90A17F0,busy,2021-05-22 09:00:00,2021-05-22 19:00:00,0
...,...,...,...,...,...
38943,C2D33F655236B21BE0536CF0D90A17F0,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
38944,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-22 09:00:00,2021-05-22 19:00:00,1
38945,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
38946,C2D33F65501CB21BE0536CF0D90A17F0,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1


In [77]:
Schedule_DF['id'].isin(Slot_DF['id']).value_counts()
#True    9564
#Name: id, dtype: int64

True    9737
Name: id, dtype: int64

In [78]:
Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')

,id_x,Line,City,State,PostalCode,id_y,actor
0,2502,581 HIGHLAND AVE.,CHESHIRE,CT,06410,C2D33F652C48B21BE0536CF0D90A17F0,2502
1,1521,9140 ROTHBURY DRIVE,GAITHERSBURG,MD,20886,C2D33F652C49B21BE0536CF0D90A17F0,1521
2,4365,709 W. COLLEGE ST.,PULASKI,TN,38478,C2D33F652C4AB21BE0536CF0D90A17F0,4365
3,1087,20 FILORS LN.,STONY POINT,NY,10980,C2D33F652C4BB21BE0536CF0D90A17F0,1087
4,4984,863 BALTIMORE PIKE,GLEN MILLS,PA,19342,C2D33F652C4CB21BE0536CF0D90A17F0,4984
...,...,...,...,...,...,...,...
9732,3634,2160 LEE RD,CLEVELAND HEIGHTS,OH,44118,C2D33F655336B21BE0536CF0D90A17F0,3634
9733,2535,49 CLAREMONT AVE.,MONTCLAIR,NJ,07042,C2D33F655337B21BE0536CF0D90A17F0,2535
9734,8875,3644 EAST OLYMPIC BOULEVARD,LOS ANGELES,CA,90023,C2D33F655338B21BE0536CF0D90A17F0,8875
9735,675,159 N. MACDADE BLVD.,GLENOLDEN,PA,19036,C2D33F655339B21BE0536CF0D90A17F0,675


In [79]:
Schedule_DF.merge(Slot_DF,how='inner',left_on='id',right_on='id')

,id,actor,Status,Start,End,ValueInteger
0,C2D33F652DCFB21BE0536CF0D90A17F0,7345,free,2021-05-22 09:00:00,2021-05-22 19:00:00,1
1,C2D33F652DCFB21BE0536CF0D90A17F0,7345,free,2021-05-23 09:00:00,2021-05-23 19:00:00,1
2,C2D33F652DCFB21BE0536CF0D90A17F0,7345,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1
3,C2D33F652DCFB21BE0536CF0D90A17F0,7345,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
4,C2D33F652DD0B21BE0536CF0D90A17F0,1482,free,2021-05-22 09:00:00,2021-05-22 19:00:00,1
...,...,...,...,...,...,...
38943,C2D33F655339B21BE0536CF0D90A17F0,675,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
38944,C2D33F65533CB21BE0536CF0D90A17F0,2004,free,2021-05-22 09:00:00,2021-05-22 19:00:00,1
38945,C2D33F65533CB21BE0536CF0D90A17F0,2004,free,2021-05-23 09:00:00,2021-05-23 19:00:00,1
38946,C2D33F65533CB21BE0536CF0D90A17F0,2004,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1


In [80]:
DF1=Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')
DF=DF1.merge(Slot_DF,how='inner',left_on='id_y',right_on='id')

In [24]:
DF=DF.drop(columns=['id_x','id_y'])

NameError: name 'DF' is not defined

In [83]:
DF.to_csv('CVS_File.csv',index=False)

# Analysis

In [50]:
DF

,Line,City,State,PostalCode,actor,id,Status,Start,End,ValueInteger
0,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2D33F652C48B21BE0536CF0D90A17F0,busy,2021-05-22 09:00:00,2021-05-22 19:00:00,0
1,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2D33F652C48B21BE0536CF0D90A17F0,busy,2021-05-23 09:00:00,2021-05-23 19:00:00,0
2,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2D33F652C48B21BE0536CF0D90A17F0,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1
3,[581 HIGHLAND AVE.],CHESHIRE,CT,06410,2502,C2D33F652C48B21BE0536CF0D90A17F0,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
4,[9140 ROTHBURY DRIVE],GAITHERSBURG,MD,20886,1521,C2D33F652C49B21BE0536CF0D90A17F0,busy,2021-05-22 09:00:00,2021-05-22 19:00:00,0
...,...,...,...,...,...,...,...,...,...,...
38943,[159 N. MACDADE BLVD.],GLENOLDEN,PA,19036,675,C2D33F655339B21BE0536CF0D90A17F0,free,2021-05-25 09:00:00,2021-05-25 19:00:00,1
38944,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2D33F65533CB21BE0536CF0D90A17F0,free,2021-05-22 09:00:00,2021-05-22 19:00:00,1
38945,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2D33F65533CB21BE0536CF0D90A17F0,free,2021-05-23 09:00:00,2021-05-23 19:00:00,1
38946,[3117 LOCKHEED BLVD.],ALEXANDRIA,VA,22306,2004,C2D33F65533CB21BE0536CF0D90A17F0,free,2021-05-24 09:00:00,2021-05-24 19:00:00,1


In [51]:
pd.pivot_table(DF,values='id',index='State',columns='Status',aggfunc='count')

Status,busy,free
State,,
AK,NaN,12.0
AL,293.0,419.0
AR,57.0,35.0
AZ,339.0,441.0
CA,2809.0,1811.0
CO,92.0,112.0
CT,635.0,81.0
DC,145.0,91.0
DE,75.0,5.0


# Refresh Data Every 15 minutes 

In [ ]:
import time

In [ ]:
# This part is to check if the number of locations changes. So I set up 15 minutes break for each loop. 
#Which took about 10 hours total. All of the output are 9737 so in total, there is only 9737 locations.

In [37]:
i=1
while i<50:
    Location_re=requests.get(df.loc['Location']["Url"])
    Schedule_re=requests.get(df.loc['Schedule']["Url"])
    Slot_re=requests.get(df.loc['Slot']["Url"])
    
    Locations_Info=[]
    for Locations in ndjson.loads(Location_re.text):
        Detail_Info={'id':Locations['id'],
                     'Line':Locations['address']['line'][0],
                     'City':Locations['address']['city'],
                     'State':Locations['address']['state'],
                     'PostalCode':Locations['address']['postalCode'],
                    }
        Locations_Info.append(Detail_Info)
    
    Schedule_Info=[]
    for Schedules in ndjson.loads(Schedule_re.text):
        Detail_Info={'id':Schedules['id'],
                     'actor':Schedules['actor'][0]['reference'].split('/')[1]
                    }
        Schedule_Info.append(Detail_Info)    
    
    Slot_Info=[]
    for Slots in ndjson.loads(Slot_re.text):
        Detail_Info={'id':Slots['schedule']['reference'].split('/')[1],
                     'Status':Slots['status'],
                     'Start':Slots['start'],
                     'End':Slots['end'],
                     'ValueInteger':Slots['extension'][1]['valueInteger']
                    }
        Slot_Info.append(Detail_Info)
    
    
    Location_DF=pd.DataFrame(Locations_Info)
    Schedule_DF=pd.DataFrame(Schedule_Info)
    Slot_DF=pd.DataFrame(Slot_Info)
    DF1=Location_DF.merge(Schedule_DF,how='left',left_on='id',right_on='actor')
    DF=DF1.merge(Slot_DF,how='inner',left_on='id_y',right_on='id')
    DF=DF.drop(columns=['id_x','id_y'])
    DF.to_csv("{}_file_at_time_{}.csv".format(i,datetime.now().time().strftime("%H_%M_%S")),index=False)

    
    print(i)
    print("There are {} different locations".format(Location_DF.shape[0]))
    time.sleep(900)
    i+=1


1
There are 9737 different locations
2
There are 9737 different locations
3
There are 9737 different locations
4
There are 9737 different locations
5
There are 9737 different locations
6
There are 9737 different locations
7
There are 9737 different locations
8
There are 9737 different locations
9
There are 9737 different locations
10
There are 9737 different locations
11
There are 9737 different locations
12
There are 9737 different locations
13
There are 9737 different locations
14
There are 9737 different locations
15
There are 9737 different locations
16
There are 9737 different locations
17
There are 9737 different locations
18
There are 9737 different locations
19
There are 9737 different locations
20
There are 9737 different locations
21
There are 9737 different locations
22
There are 9737 different locations
23
There are 9737 different locations
24
There are 9737 different locations
25
There are 9737 different locations
26
There are 9737 different locations
27
There are 9737 dif

In [89]:
datetime.now()

datetime.datetime(2021, 5, 21, 3, 6, 3, 396087)

In [116]:
"DF_at {}".format(datetime.now().time().strftime("%H:%M"))

'DF_at   03:12'

In [40]:
print("There are {} different locations".format(Location_DF.shape[0]))

There are 9737 different locations
